# PathManager 테스트 노트북

이 노트북은 PathManager 클래스의 기능을 테스트하고 검증합니다.

In [ ]:
import sys
from pathlib import Path

# 상위 디렉토리의 code를 import 경로에 추가
sys.path.append(str(Path.cwd().parent / 'code'))

from utils.path_utils import PathManager, path_manager, get_project_root, resolve_path, ensure_dir

## 1. 프로젝트 루트 감지 테스트

In [ ]:
# 프로젝트 루트 확인
project_root = get_project_root()
print(f"프로젝트 루트: {project_root}")
print(f"절대 경로: {project_root.absolute()}")

# 프로젝트 구조 확인
print("\n프로젝트 구조:")
for item in project_root.iterdir():
    if item.is_dir():
        print(f"  📁 {item.name}/")
    else:
        print(f"  📄 {item.name}")

## 2. 경로 해결 테스트

In [ ]:
# 상대 경로를 절대 경로로 변환
test_paths = [
    "data/train.csv",
    "config/config.yaml",
    "outputs/experiment_1/results.json",
    "models/best_model/pytorch_model.bin"
]

print("상대 경로 → 절대 경로 변환:")
for rel_path in test_paths:
    abs_path = resolve_path(rel_path)
    print(f"  {rel_path} → {abs_path}")
    print(f"    존재 여부: {abs_path.exists()}")

## 3. 디렉토리 생성 테스트

In [ ]:
# 테스트용 디렉토리 생성
test_dirs = [
    "test_output/experiment_1",
    "test_models/transformer_v1",
    "test_logs/2024_01"
]

print("디렉토리 생성 테스트:")
for dir_path in test_dirs:
    created_path = ensure_dir(dir_path)
    print(f"  생성됨: {created_path}")
    print(f"    상대 경로: {path_manager.relative_to_root(created_path)}")

## 4. 특수 경로 메소드 테스트

In [ ]:
# 데이터 경로
data_dir = path_manager.get_data_path()
train_data = path_manager.get_data_path("train.csv")
print(f"데이터 디렉토리: {data_dir}")
print(f"학습 데이터: {train_data}")

# 설정 경로
config_dir = path_manager.get_config_path()
main_config = path_manager.get_config_path("config.yaml")
print(f"\n설정 디렉토리: {config_dir}")
print(f"메인 설정: {main_config}")

# 실험 출력 경로
exp_name = "test_experiment"
output_dir = path_manager.get_output_path(exp_name)
results_file = path_manager.get_output_path(exp_name, "results.json")
print(f"\n실험 출력 디렉토리: {output_dir}")
print(f"결과 파일: {results_file}")

# 모델 경로
model_name = "bert_summarizer"
model_dir = path_manager.get_model_path(model_name)
weights_file = path_manager.get_model_path(model_name, "pytorch_model.bin")
print(f"\n모델 디렉토리: {model_dir}")
print(f"가중치 파일: {weights_file}")

# 로그 경로
log_dir = path_manager.get_log_path(exp_name)
log_file = path_manager.get_log_path(exp_name, "training.log")
print(f"\n로그 디렉토리: {log_dir}")
print(f"로그 파일: {log_file}")

## 5. 크로스 플랫폼 테스트

In [ ]:
import platform
import os

# 현재 플랫폼 정보
print(f"운영체제: {platform.system()}")
print(f"플랫폼: {platform.platform()}")
print(f"경로 구분자: '{os.sep}'")

# 경로 구분자 처리 테스트
test_path = "data/processed/train.csv"
resolved = resolve_path(test_path)
print(f"\n입력 경로: {test_path}")
print(f"해결된 경로: {resolved}")
print(f"정규화된 경로: {resolved.as_posix()}")

## 6. 실제 사용 예제

In [ ]:
# ConfigManager에서 사용
from utils.config_manager import ConfigManager

config_manager = ConfigManager()
print(f"ConfigManager 기본 디렉토리: {config_manager.base_dir}")

# DataProcessor에서 사용
from utils.data_utils import DataProcessor

# 데이터 파일 경로 (상대 경로로 지정)
data_file = "data/sample.csv"
print(f"\n데이터 파일 상대 경로: {data_file}")
print(f"해결된 절대 경로: {resolve_path(data_file)}")

## 7. 정리 - 테스트용 디렉토리 삭제

In [ ]:
import shutil

# 테스트용으로 생성한 디렉토리 삭제
test_root_dirs = ["test_output", "test_models", "test_logs"]

for dir_name in test_root_dirs:
    dir_path = resolve_path(dir_name)
    if dir_path.exists():
        shutil.rmtree(dir_path)
        print(f"삭제됨: {dir_path}")

## 결론

PathManager는 다음과 같은 기능을 제공합니다:

1. **자동 프로젝트 루트 감지**: code/, config/ 디렉토리 또는 .git, pyproject.toml을 기준으로 프로젝트 루트를 찾습니다.
2. **상대 경로 해결**: 모든 상대 경로를 프로젝트 루트 기준 절대 경로로 변환합니다.
3. **디렉토리 자동 생성**: ensure_dir()로 필요한 디렉토리를 자동 생성합니다.
4. **특수 경로 메소드**: 데이터, 설정, 출력, 모델, 로그 등의 표준 경로를 쉽게 얻을 수 있습니다.
5. **크로스 플랫폼 지원**: Windows, Mac, Linux에서 동일하게 동작합니다.

이제 프로젝트를 다른 경로로 이동하거나 다른 시스템에서 실행해도 경로 문제가 발생하지 않습니다!